In [57]:
import yfinance as yf
import datetime as dt
import pandas as pd
import numpy as np
from numpy import arange
import matplotlib.pyplot as plt
from pandas import read_csv

import plotly.express as px
import plotly.graph_objects as go

from statsmodels.tsa.seasonal import seasonal_decompose


from tsmoothie.utils_func import sim_randomwalk
from tsmoothie.smoother import *

from sklearn.preprocessing import MinMaxScaler

from ta import add_all_ta_features
from ta.utils import dropna
import ta as ta


def GetTriario(base,pulo=5):
    
    de = 0
    ate = pulo
    
    df = base.tail(len(base)-(len(base)%pulo))

    df = df.reset_index()

    lista_diario = list()
    
    for i in range(int(len(df)/pulo)):

        lista_diario.append(df[de:ate])

        de += pulo
        ate += pulo
    
    
    abertura = list()
    fechamento = list()
    maximo = list()
    minimo = list()
    datas = list()

    i = 0
    for triario in lista_diario:

        datas.append(triario["Date"][i])
        abertura.append(triario["Open"][i])
        fechamento.append(triario["Close"][i])
        maximo.append(triario["High"].max())
        minimo.append(triario["Low"].min())

        i+=pulo
        
        
    triario = zip(datas,abertura,fechamento,minimo,maximo)
    
    triario_df = pd.DataFrame(triario,columns=["Date","Open","Close","Low","High"])
    
    triario_df = triario_df.set_index("Date")

    return triario_df

In [47]:
ticker ="PETR3.SA"

df = yf.download(ticker)

df = df["2015":]

df_tri = GetTriario(df,5)


[*********************100%***********************]  1 of 1 completed


In [38]:
#REPRESENTAÇÃO DO DF_TRI

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        name="Ação",
        x=df_tri["Date"],
        y=df_tri["Close"],
        line=dict(color='blue', width=1))
)

In [52]:
#REPRESENTAÇÃO DO DF NORMAL COM TODAS AS INFORMAÇÕES

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        name="Ação",
        x=df.index,
        y=df["Close"],
        line=dict(color='blue', width=1))
)

In [84]:
#REMOÇÃO DA TENDÊNCIA DO DF_TRI

res = seasonal_decompose(df_tri.Close, model='multiplicative', extrapolate_trend='freq',period=50)

detrended = df_tri.Close.values - res.trend

detrended_df = pd.DataFrame(detrended)


tsi_np = detrended_df["trend"].to_numpy()

tsi_np = tsi_np.reshape(1,len(tsi_np))

smoother_tsi = GaussianSmoother(n_knots=130, sigma=0.00009)
smoother_tsi.smooth(tsi_np)


fig = go.Figure()


fig.add_trace(
    go.Scatter(
        name="Fechamento sem tendência sazonal",
        x=detrended_df.index,
        y=detrended_df["trend"],
        line=dict(color='blue', width=1))
)


fig.add_trace(
    go.Scatter(
        x=detrended_df.tail(len(detrended_df)).index,
        y=smoother_tsi.smooth_data[0],
        mode='lines',
        name='Gaussian',
        line=dict(color="orange"))
)


fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')

In [86]:
#REMOÇÃO DA TENDÊNCIA DO DF NORMAL. REPARE A QUANTIDADE MUITO MAIOR DE INFORMAÇÃO.
#PORÉM ESSE GRÁFICO MOSTRA QUE ELE CONSEGUE ANALISAR AS TENDÊNCIAS DE CURTO PRAZO, MUITO BOM PARA INVESTIMENTOS CURTOS.


res = seasonal_decompose(df.Close, model='multiplicative', extrapolate_trend='freq',period=50)

detrended = df.Close.values - res.trend

detrended_df = pd.DataFrame(detrended)


tsi_np = detrended_df["trend"].to_numpy()

tsi_np = tsi_np.reshape(1,len(tsi_np))

smoother_tsi = GaussianSmoother(n_knots=130, sigma=0.00009)
smoother_tsi.smooth(tsi_np)


fig = go.Figure()


fig.add_trace(
    go.Scatter(
        name="Fechamento sem tendência sazonal",
        x=detrended_df.index,
        y=detrended_df["trend"],
        line=dict(color='blue', width=1))
)


fig.add_trace(
    go.Scatter(
        x=detrended_df.tail(len(detrended_df)).index,
        y=smoother_tsi.smooth_data[0],
        mode='lines',
        name='Gaussian',
        line=dict(color="orange"))
)


fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')

In [88]:
#TSI

tsi_config=[25,13]
resultados_tsi = ta.momentum.TSIIndicator(df_tri["Close"],tsi_config[0],tsi_config[1],False)

resultados_tsi.tsi().dropna(inplace=True)


#Gaussian TSI
resultados_tsi_sem_null = resultados_tsi.tsi().dropna()

tsi_np = resultados_tsi_sem_null.to_numpy()

tsi_np = tsi_np.reshape(1,len(tsi_np))


smoother_tsi = GaussianSmoother(n_knots=60, sigma=0.001)
smoother_tsi.smooth(tsi_np)


fig = go.Figure()

fig.add_trace(
    go.Scatter(
        name="TSI",
        x=df_tri.tail(len(resultados_tsi_sem_null)).index,
        y=resultados_tsi_sem_null,
        line=dict(color='blue', width=1))
)

fig.add_trace(
    go.Scatter(
        x=df_tri.tail(len(resultados_tsi_sem_null)).index,
        y=smoother_tsi.smooth_data[0],
        mode='lines',
        name='TSI - Gaussian',
        line=dict(color="orange"))
)

fig.add_hline(y=0,line_color="blue",line_width=1, line_dash="dash")
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')

In [89]:
#TSI

tsi_config=[25,13]
resultados_tsi = ta.momentum.TSIIndicator(df["Close"],tsi_config[0],tsi_config[1],False)

resultados_tsi.tsi().dropna(inplace=True)


#Gaussian TSI
resultados_tsi_sem_null = resultados_tsi.tsi().dropna()

tsi_np = resultados_tsi_sem_null.to_numpy()

tsi_np = tsi_np.reshape(1,len(tsi_np))


smoother_tsi = GaussianSmoother(n_knots=60, sigma=0.001)
smoother_tsi.smooth(tsi_np)


fig = go.Figure()

fig.add_trace(
    go.Scatter(
        name="TSI",
        x=df.tail(len(resultados_tsi_sem_null)).index,
        y=resultados_tsi_sem_null,
        line=dict(color='blue', width=1))
)

fig.add_trace(
    go.Scatter(
        x=df.tail(len(resultados_tsi_sem_null)).index,
        y=smoother_tsi.smooth_data[0],
        mode='lines',
        name='TSI - Gaussian',
        line=dict(color="orange"))
)

fig.add_hline(y=0,line_color="blue",line_width=1, line_dash="dash")
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')